# Overview of DeepBugs replication and comparison with using CNN architecture for swapped argument bug pattern

## using google colab pro high-ram machine with T4 GPU

following readme on github page
* All commands are called from the main directory.
* Python code (most of the implementation) and JavaScript code (for extracting data from .js files) are in the `/python` and `/javascript` directories.
* All data to learn from, e.g., .js files are expected to be in the `/data` directory.
* All data that is generated, e.g., intermediate representations, are written into the main directory. It is recommended to move them into separate directories.
* All generated data files have a timestamp as part of the file name. Below, all files are used with `*`. When running commands multiple times, make sure to use the most recent files.

# Clone github for DeepBugs
I forked the project to my own github in order to corrections to the code.
Certain keras imports are depricated causing errors.

* depricated import

`from tensorflow.python.keras.models import Sequential`

`from tensorflow.python.keras.layers.core import Dense, Dropout`


* correct import statement

`from keras.models import Sequential`

`from keras.layers import Dense, Dropout`

* added CNN architecture to the model

In [ ]:
# Running code from the "working" branch
! git clone -b working https://github.com/livingdan/DeepBugs_replication

Cloning into 'DeepBugs_replication'...
remote: Enumerating objects: 502, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 502 (delta 89), reused 116 (delta 84), pack-reused 372
Receiving objects: 100% (502/502), 258.44 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (283/283), done.
Updating files: 100% (134/134), done.


In [ ]:
# move contents of DeepBugs to main directory
! mv DeepBugs_replication/* .

In [ ]:
# install dependencies npm modules acorn, estraverse, walk-sync
# npm is a package manager for the JavaScript programming language
! npm install acorn
! npm install estraverse
! npm install walk-sync

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ acorn@8.11.3
added 1 package and audited 1 package in 0.429s
found 0 vulnerabilities

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ estraverse@5.3.0
added 1 package and audited 2 packages in 0.443s
found 0 vulnerabilities



   ╭────────────────────────────────────────────────────────────────╮
   │                                                            

# 1. Download Training and Testing datasets
* The full corpus can be downloaded [here](http://www.srl.inf.ethz.ch/js150.php) and is expected to be stored in `data/js/programs_all`. It consists of 100.000 training files, listed in `data/js/programs_training.txt`, and 50.000 files for validation, listed in `data/js/programs_eval.txt`.


In [ ]:
! gdown http://files.srl.inf.ethz.ch/data/js_dataset.tar.gz

Downloading...
From: http://files.srl.inf.ethz.ch/data/js_dataset.tar.gz
To: /content/js_dataset.tar.gz
100% 2.63G/2.63G [02:17<00:00, 19.2MB/s]


In [ ]:
! tar -xzf js_dataset.tar.gz
#extracts

In [ ]:
! mkdir data/js/programs_all
! tar -xzf data.tar.gz -C data/js/programs_all
#makes directory and loads the data there

In [ ]:
! mv data/js/programs_all/data/* data/js/programs_all
#move data to the new location

# 2. Embeddings for Identifiers using Word2Vec

The above bug detector rely on a vector representation for identifier names and literals. To use our framework, the easiest is to use the shipped `token_to_vector.json` file. Alternatively, you can learn the embeddings via Word2Vec as follows:

1. Extract identifiers and tokens:

`node javascript/extractFromJS.js tokens --parallel 4 data/js/programs_50_training.txt data/js/programs_50`

  * The command produces `tokens_*.json` files.
  
2. Encode identifiers and literals with context into arrays of numbers (for faster reading during learning):
  
  `python3 python/TokensToTopTokens.py tokens_*.json`
  
  * The arguments are the just created files.
  * The command produces `encoded_tokens_*.json` files and a file `token_to_number_*.json` that assigns a number to each identifier and literal.

3. Learn embeddings for identifiers and literals:
  
  `python3 python/EmbeddingLearnerWord2Vec.py token_to_number_*.json encoded_tokens_*.json`

  * The arguments are the just created files.
  * The command produces a file `token_to_vector_*.json`.

In [ ]:
# Extract identifiers and tokens from all dataset
!node javascript/extractFromJS.js tokens --parallel 4 data/js/programs_all.txt data/js/programs_all

internal/modules/cjs/loader.js:883
  throw err;
  ^

Error: Cannot find module '/content/javascript/extractFromJS.js'
    at Function.Module._resolveFilename (internal/modules/cjs/loader.js:880:15)
    at Function.Module._load (internal/modules/cjs/loader.js:725:27)
    at Function.executeUserEntryPoint [as runMain] (internal/modules/run_main.js:72:12)
    at internal/main/run_main_module.js:17:47 {
  code: 'MODULE_NOT_FOUND',
  requireStack: []
}


In [ ]:
# move tokens to own directory
!mkdir tokens/
!mv tokens_*.json tokens/

In [ ]:
# Encode identifiers and literals with context into arrays of numbers for training
! python3 python/TokensToTopTokens.py tokens/tokens_*.json

Total files: 692
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616239890.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616247151.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616254447.json
Reading file /content/tokens/tokens_1712616255120.json
Worker starting to read 23 files
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616263479.json
Reading file /content/tokens/tokens_1712616269754.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616276051.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616280561.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616294204.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616300773.json
Worker starting to read 23 files
Reading file /content/tokens/tokens_1712616307757.json
Worker starting to read 23 files
Reading file /c

In [ ]:
# move encoded tokens to own directory
!mkdir encoded_tokens/
!mv encoded_tokens_*.json encoded_tokens/
!mv token_to_number_*.json encoded_tokens/

In [ ]:
# Learn embeddings for identifiers and literals training
! python3 python/EmbeddingLearnerWord2Vec.py encoded_tokens/token_to_number_*.json encoded_tokens/encoded_tokens_*.json

/content/python/EmbeddingLearnerWord2Vec.py:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(data_paths) is 0:
Reading file /content/encoded_tokens/encoded_tokens_1712616521675.json
Reading file /content/encoded_tokens/encoded_tokens_1712616523314.json
Reading file /content/encoded_tokens/encoded_tokens_1712616523846.json
Reading file /content/encoded_tokens/encoded_tokens_1712616523996.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524032.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524065.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524161.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524164.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524193.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524242.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524272.json
Reading file /content/encoded_tokens/encoded_tokens_1712616524407.json
Reading file /cont

In [ ]:
# move token_to_vector file to own directory
!mkdir token_to_vector/
!mv token_to_vector_1*.json token_to_vector/


# 3. Learning a Bug Detector
Creating a bug detector consists of two main steps:

1. Extract positive (i.e., likely correct) and negative (i.e., likely buggy) training examples from code.

2. Train a classifier to distinguish correct from incorrect code examples.

This replication example will address the swapped argument bug detector
* The `SwappedArgs` bug detector looks for accidentally swapped arguments of a function call, e.g., calling `setPoint(y,x)` instead of `setPoint(x,y)`.

## 3.1 Extract positive and Negative training examples

`node javascript/extractFromJS.js calls --parallel 4 data/js/programs_50_training.txt data/js/programs_50`

  * The `--parallel` argument sets the number of processes to run.
  * `programs_50_training.txt` contains files to include (one file per line). To extract data for validation, run the command with `data/js/programs_50_eval.txt`.
  * The last argument is a directory that gets recursively scanned for .js files, considering only files listed in the file provided as the second argument.
  * The command produces `calls_*.json` files, which is data suitable for the `SwappedArgs` bug detector. For the other bug two detectors, replace `calls` with `binOps` in the above command.

### Unzip calls previously extrated to speed up model

In [ ]:
# unzip previously extracted calls to speed up process for training/eval
!unzip swapped_calls/training.zip
!unzip swapped_calls/eval.zip

Archive:  swapped_calls/training.zip
   creating: training/
  inflating: training/calls_1713390794262.json  
  inflating: training/calls_1713392938965.json  
  inflating: training/calls_1713393002173.json  
  inflating: training/calls_1713392197326.json  
  inflating: training/calls_1713392972421.json  
  inflating: training/calls_1713392775709.json  
  inflating: training/calls_1713391538019.json  
  inflating: training/calls_1713391020090.json  
  inflating: training/calls_1713392756167.json  
  inflating: training/calls_1713391691348.json  
  inflating: training/calls_1713392121616.json  
  inflating: training/calls_1713390841707.json  
  inflating: training/calls_1713391226074.json  
  inflating: training/calls_1713390945978.json  
  inflating: training/calls_1713392956256.json  
  inflating: training/calls_1713390873511.json  
  inflating: training/calls_1713391516475.json  
  inflating: training/calls_1713392719598.json  
  inflating: training/calls_1713391162740.json  
  inflati

### Full Corpus data set
For google colab the training set it too large to use using high-ram T4 machine.

In [ ]:
# cut the training dataset by 10% as loading full dataset for model causes out of memory error
import random

with open("data/js/programs_training.txt", "r") as f:
    lines = f.readlines()

# Shuffle the list of lines
print(len(lines))
random.shuffle(lines)

# Calculate the number of lines to remove (10%)
num_lines_to_remove = int(len(lines) * 0.1)

# Remove the specified number of lines
for i in range(num_lines_to_remove):
    lines.pop()

print(len(lines))
# Write the remaining lines back to the file
with open("data/js/programs_training.txt", "w") as f:
    f.writelines(lines)

100000
90000


In [ ]:
# Using full dataset training data
! node javascript/extractFromJS.js calls --parallel 4 data/js/programs_training.txt data/js/programs_all
! mkdir training/
! mv calls_* training/

Streaming output truncated to the last 5000 lines.
Added calls. Total now: 723

Reading data/js/programs_all/BinaryMuse/planetary.js/site/public/semantic/javascript/semantic.js

Reading data/js/programs_all/BriteSnow/snow/src/main/resources/com/britesnow/snow/web/less/less-rhino-1.2.2.js

Added calls. Total now: 2623

Reading data/js/programs_all/BrowserSync/UI/lib/plugins/connections/connections.client.js

Added calls. Total now: 2625

Reading data/js/programs_all/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/src-noconflict/mode-jade.js

Reading data/js/programs_all/BrowserSync/UI/lib/plugins/overview/overview.client.js

Added calls. Total now: 2625

Reading data/js/programs_all/BrowserSync/UI/lib/plugins/remote-debug/latency/latency.client.js

Added calls. Total now: 2625

Reading data/js/programs_all/BrowserSync/UI/lib/plugins/remote-debug/overlay-grid/overlay-grid.client.js

Added calls. Total now: 2625

Reading data/js/programs_all/BrowserSync/UI/lib/plugins/remote-debug/

In [ ]:
# Using full dataset evaluation data
! node javascript/extractFromJS.js calls --parallel 4 data/js/programs_eval.txt data/js/programs_all
! mkdir eval/
! mv calls_* eval/

Streaming output truncated to the last 5000 lines.

Added calls. Total now: 183

Reading data/js/programs_all/Breeze/breeze.js.samples/node/zza-mean/public/test/specs.async/api.breeze.async.spec.js

Added calls. Total now: 221

Reading data/js/programs_all/Chorus/chorus/spec/javascripts/views/style_guide/colore_palette_spec.js

Added calls. Total now: 183

Reading data/js/programs_all/Chorus/chorus/spec/javascripts/views/users/user_sidebar_spec.js

Reading data/js/programs_all/Breeze/breeze.js.samples/ruby/CCJS-Ruby/client/app/services/breeze.ajaxrestinterceptor.js

Added calls. Total now: 223

Reading data/js/programs_all/Breeze/breeze.js.samples/ruby/CCJS-Ruby/client/app/services/breeze.jsonResultsAdapter.js

Added calls. Total now: 224

Added calls. Total now: 950

Added calls. Total now: 189

Reading data/js/programs_all/Breeze/breeze.js.samples/ruby/CCJS-Ruby/client/app/viewmodels/shell.js

Added calls. Total now: 227

Reading data/js/programs_all/DDMAL/Neon.js/src/pageview.js

Ad

## 3.2 Train a classifier to identify bugs
1. Train and validate the classifier Original model and CNN model on same data preparations

`python3 python/BugLearnAndValidate.py --pattern SwappedArgs --token_emb token_to_vector.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --training_data calls_xx*.json --validation_data calls_yy*.json`

  * The first argument selects the bug pattern.
  * The next three arguments are vector representations for tokens (here: identifiers and literals), for types, and for AST node types. These files are provided in the repository.
  * The remaining arguments are two lists of .json files. They contain the training and validation data extracted in Step 1.
  * After learning the bug detector, the command measures accurracy and recall w.r.t. seeded bugs and writes a list of potential bugs in the unmodified validation code (see `poss_anomalies.txt`).

### using previously learned by me embeddings token_to_vector_Replication.json to speed up notebook


In [ ]:
# Train and validate the classifier original implementation
! python3 python/BugLearnAndValidate.py --pattern SwappedArgs --token_emb token_to_vector_Replication.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --training_data training/calls_*.json --validation_data eval/calls_*.json

2024-04-18 18:30:27.026484: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 18:30:27.026547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 18:30:27.028048: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 18:30:27.036271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 18:30:28.428638: W tensorflow/compiler/tf2

In [ ]:
# Train and validate the classifier CNN implementation
! python3 python/BugLearnAndValidateCNN.py --pattern SwappedArgs --token_emb token_to_vector_Replication.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --training_data training/calls_*.json --validation_data eval/calls_*.json

2024-04-18 17:36:17.725083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 17:36:17.725146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 17:36:17.726490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 17:36:17.733598: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 17:36:18.711854: W tensorflow/compiler/tf2

# 4. Finding Bugs
Finding bugs in one or more source files consists of these two steps:
1. Extract code pieces
2. Use a trained classifier to identify bugs

In [ ]:
# Extract code peices from file directory
#! node javascript/extractFromJS.js calls --files <list of files>
! node javascript/extractFromJS.js calls --files data/js/programs_50/*.js
! mkdir find_bugs/
! mv calls_* find_bugs/

Use a trained classifier to identify bugs

`python3 python/BugFind.py --pattern SwappedArgs --threshold 0.95 --model some/dir --token_emb token_to_vector.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --testing_data calls_xx*.json`

  * The first argument selects the bug pattern.
  * 0.95 is the threshold for reporting bugs; higher means fewer warnings of higher certainty.
  * --model sets the directory to load a trained model from.
  * The next three arguments are vector representations for tokens (here: identifiers and literals), for types, and for AST node types. These files are provided in the repository.
  * The remaining argument is a list of .json files. They contain the data extracted in Step 1.
  * The command examines every code piece and writes a list of potential bugs with its probability of being incorrect

In [ ]:
# Use a trained classifier to identify bugs
! python3 python/BugFind.py --pattern SwappedArgs --threshold 0.95 --model bug_detection_model_*/ --token_emb token_to_vector.json --type_emb type_to_vector.json --node_emb node_type_to_vector.json --testing_data eval/calls_*.json

2024-04-08 23:33:19.833229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 23:33:19.833280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 23:33:19.834666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 23:33:19.841789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 23:33:20.861210: W tensorflow/comp